In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [14]:
def prepare_sequence(seq, to_index):
    idxs = [to_index[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

training_data = [
    ("The dog ate the apple".split(), ["D", "N", "V", "D", "N"]),
    ("Everybody read that book".split(), ["N", "V", "D", "N"]),
]

In [5]:
training_data

[(['The', 'dog', 'ate', 'the', 'apple'], ['D', 'N', 'V', 'D', 'N']),
 (['Everybody', 'read', 'that', 'book'], ['N', 'V', 'D', 'N'])]

In [6]:
word_to_index = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_index:
            word_to_index[word] = len(word_to_index)

In [7]:
word_to_index

{'The': 0,
 'dog': 1,
 'ate': 2,
 'the': 3,
 'apple': 4,
 'Everybody': 5,
 'read': 6,
 'that': 7,
 'book': 8}

In [8]:
tag_to_index = {"D": 0, "N": 1, "V": 2}
index_to_tag = ["D", "N", "V"]

In [9]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [12]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_index), len(tag_to_index))

In [13]:
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_index)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(300):
    for sentence, tags in training_data:
        model.zero_grad()

        sentence_in = prepare_sequence(sentence, word_to_index)
        targets = prepare_sequence(tags, tag_to_index)

        tag_scores = model(sentence_in)

        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_index)
    tag_scores = model(inputs)
    print(tag_scores)

NameError: name 'prepare_sequence' is not defined